# SISTEMAS MULTIMEDIA - PROYECTO OPENROUTESERVICE
## INTRODUCCIÓN
En este notebook se explicará qué es y cómo se ha implementado nuestro proyecto para la asignatura de Sistemas Multimedia.

El objetivo de este proyecto es ayudar a los estudiantes de Cáceres a elegir su piso proporcionándoles información que otras páginas web o aplicaciones no ofrecen. Por ejemplo, qué puntos de interés (supermercados, farmacias, bancos, etc.) se encuentran cerca de cada piso. Para ello, se desarrollará un bot de telegram que haga uso de las funcionalidades de la API de openrouteservice. A través del cual, los usuarios podrán elegir una serie de preferencias y realizar búsquedas de pisos en función de éstas.

##PREPARACIÓN DEL ENTORNO

Antes que nada, para poder ejecutar el código de este notebook, se deberán instalar las librerías de openrouteservice y pyTelegramBotAPI. Los comandos necesarios para este paso se encuentran a continuación.

In [1]:
!pip install openrouteservice 
!pip install pyTelegramBotAPI
!pip install git+https://github.com/eternnoir/pyTelegramBotAPI.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 27.9 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.7.0-py3-none-any.whl size=192826 sha256=92e68659bf8ef1c4d2fb172811e3320248d89388f6c19e2814e0ecbaed79616c
  Stored in directory: /root/.cache/pip/wheels/ba/13/da/8abf941f7cf9f993cde6118a56a7c24e12ed791507acd8ea39
Successfully built pyTelegramBotAPI
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/eternnoir/pyTelegramBotAPI.git to /tmp/pip-req-build-hucsi93v
  Running command git clone -q https://github.com/eternnoir/pyTelegramBotAPI.git /tmp/pip-req-build-hucsi93v


Una vez instalado todo lo necesario, se puede seguir ejecutando el resto del proyecto

##CÓDIGO DEL PROYECTO
###CLAVES DE LA API OPENROUTESERVICE

Para utilizar las funcionalidades de la API de openrouteservice, fue necesario registrarse en su página web y crear una clave única para incluirla en nuestro proyecto. Esta clave tiene un límite de peticiones diarias, tras las cuales se bloquea. Por lo que cada integrante del grupo tiene su propia clave para poder intercambiarla cada vez que se bloquee una de ellas.

###DICCIONARIOS
A falta de una base de datos para guardar la información de los pisos, los puntos de interés, las preferencias de cada usuario, etc., se ha optado por implementar una serie de diccionarios que servirán para guardar esta información y relacionarla entre sí.

* **preferencias**: almacena qué puntos de interés quiere buscar o no el usuario.
* **pisos**: almacena la lista de pisos disponibles.
* **params_iso**: guarda los parámetros que se utilizarán para la búsqueda de puntos de interés alrededor de cada piso
* **parametros_pi**: almacena los parámetros que se utilizarán para realizar las peticiones de puntos de interés a la API.
* **categorias_pi**: almacena los nombres de las categorías de puntos interés y su código según están recogidas en la API.
* **iconos**: almacena el nombre de los iconos correspondientes a los puntos de interés según la librería de iconos *FontAwesome*.
* **colores**: almacena la lista de colores disponibles según la API de *Folium*.
* **parametros_ruta**: almacena los parámetros que se utilizarán para dibujar las rutas entre los pisos y los puntos de interés.

In [2]:
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
import telebot
import folium
import random
import operator
import numpy as np
import os
from openrouteservice import client

# Claves de la API
# Álvaro: 5b3ce3597851110001cf6248dadf84031f594fb89b6b2b002aad8479
# Abel: 5b3ce3597851110001cf6248c8ec279dee31452aa50cb3c9bf791464
# Julia: 5b3ce3597851110001cf6248b9288437ee8d45b085f193edfd1a771c
api_key = '5b3ce3597851110001cf6248c8ec279dee31452aa50cb3c9bf791464'
ors = client.Client(key=api_key)

# Lista de preferencias del usuario
preferencias = {"supermarket": 0,
                "platform": 0,  # Paradas de bus
                "bus_station": 0,
                "hairdresser": 0,
                "pub": 0,
                "bar": 0,
                "fast-food": 0,
                "pharmacy": 0,
                "fitness_centre": 0,
                "park": 0,
                "bank": 0,
                "hospital": 0,
                "university": 0
                }

# Lista de pisos
pisos = {'Piso 1': {'location': [39.4727, -6.3713]},
         'Piso 2': {'location': [39.4677, -6.3775]},
         'Piso 3': {'location': [39.4713, -6.3885]},
         'Piso 4': {'location': [39.4626, -6.3674]},
         'Piso 5': {'location': [39.4596, -6.3774]}
         }

# Parametros de las zonas de alrededor de los pisos
params_iso = {'profile': 'foot-walking',
              'range': [5 * 60] # Limites de la zona a los que llegas en X minutos
              }

# Parámetros de las peticiones
parametros_pi = {'request': 'pois',
                 'sortby': 'distance'
                 }

# Tipos de puntos de interes según:
# https://giscience.github.io/openrouteservice/documentation/Places.html
categorias_pi = {'supermarket': [518],
                 'platform': [607],
                 'bus_station': [587],
                 'hairdresser': [395],
                 'pub': [569],
                 'bar': [561],
                 'fast-food': [566],
                 'pharmacy': [208],
                 'fitness_centre': [288],
                 'park': [272],
                 'bank': [192],
                 'hospital': [206],
                 'university': [157]
                 }

# Iconos de los puntos de interés
iconos = {'supermarket': 'shopping-cart',
          'platform': 'bus',
          'bus_station': 'map-signs',
          'hairdresser': 'scissors',
          'pub': 'music',
          'night-club': 'music',
          'bar': 'beer',
          'fast-food': 'cutlery',
          'pharmacy': 'medkit',
          'fitness_centre': 'futbol-o',
          'park': 'leaf',
          'bank': 'eur',
          'hospital': 'hospital-o',
          'university': 'graduation-cap'
          }

# Lista de colores disponibles
colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen',
           'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']


# Parámetros de las rutas
parametros_ruta = {'profile': 'foot-walking',
                   'format_out': 'geojson',
                   'geometry': 'true',
                   'format': 'geojson',
                   'instructions': 'false',
                   'radiuses': 10000
                   }  


###BÚSQUEDA DE PISOS
En la función **buscar_pisos(...)** se implementa el código necesario para realizar la búsqueda de pisos en función de las preferencias y parámetros fijados anteriormente.
Para empezar, crean y se dibujan en el mapa las zonas en las que se buscarán puntos de interés alrededor de cada piso. Después, se eliminan del diccionario de preferencias aquellas que el usuario no ha elegido, guardando previamente una copia para restaurar el diccionario original tras la búsqueda. Seguidamente, se piden los puntos de interés a la API, se cuenta el número de categorías de puntos de interés distintas por cada piso y se ordenan estos pisos según ese número. Acto seguido, se piden las rutas y se dibujan en el mapa. También se calculan las duraciones de cada ruta. Después se pintan en el mapa todos los puntos de interés, destacando de un color distinto el punto de interés más cercano de cada categoría. Y, para finalizar, se pintan en el mapa los pisos, destacando en otro color aquel que más categorías reúne en su área y se genera un archivo lamado "*mapa_pisos.html*" que contiene el mapa final con toda la información solicitada.

In [3]:
# Buscar pisos y puntos de interés en función de las preferencias del usuario
def buscar_pisos(preferencias, pisos, params_iso, parametros_pi, categorias_pi, iconos, parametros_ruta):
   # Inicializa el mapa
    mapa = folium.Map(tiles='Stamen Toner', location=([39.46832, -6.3793]), zoom_start=14)

    # Por cada piso, se dibuja alrededor la zona en la que se buscarán los puntos de interés
    for nombre, piso in pisos.items():
        params_iso['locations'] = [list(reversed(piso['location']))]
        piso['iso'] = ors.isochrones(**params_iso)
        folium.features.GeoJson(piso['iso'], style_function=lambda x: {'fillColor': random.choice(colores),
                                                                       'color': '#676b70',
                                                                       'fillOpacity': 0.3}
                                ).add_to(mapa)

    # Elimina del diccionario de puntos de interes aquellos que no quiere el usuario
    # Se hace una copia para restaurarlo después de la búsqueda
    categorias_pi_copia = dict(categorias_pi)
    for categoria, pref in preferencias.items():
        if pref == 0:
            del categorias_pi[categoria]

    # Guarda el número de puntos de interes distintos por cada piso
    categorias_por_piso = {}
    for nombre, piso in pisos.items():
        # Guarda las categorías en un diccionario para acceder fácilmente
        piso['categories'] = dict()
        parametros_pi['geojson'] = piso['iso']['features'][0]['geometry']
        print(nombre)

        # Calcular el número de puntos de interés distintos
        num_categorias = 0
        for tipo, categoria in categorias_pi.items():
            parametros_pi['filter_category_ids'] = categoria
            piso['categories'][tipo] = dict()
            piso['categories'][tipo]['geojson'] = ors.places(**parametros_pi)['features']  # Petición de punto de interés actual

            cat_total = len(piso['categories'][tipo]['geojson'])
            print(f"\t{tipo}: {cat_total}")

            # Si hay algún punto de interes suma 1 a num_categorias
            if cat_total > 0:
                num_categorias += 1

        # Añade el piso y el numero de categorias distintas pertenecientes al piso
        categorias_por_piso[nombre] = num_categorias

    # Ordenar pisos según cuántos puntos de interés diferentes contiene
    categorias_por_piso = sorted(categorias_por_piso.items(), key=operator.itemgetter(1), reverse=True)
    
    # Por cada piso se obtinenen las rutas a los puntos de interés
    for piso in pisos.values():
        for cat, pois in piso['categories'].items():
            pois['durations'] = []
            for poi in pois['geojson']:
                poi_coords = poi['geometry']['coordinates']

                # Guarda las coordenadas de la ruta
                parametros_ruta['coordinates'] = [list(reversed(piso['location'])),
                                                  poi_coords
                                                  ]
                # Obtiene la ruta
                json_route = ors.directions(**parametros_ruta)
                
                # Dibuja la ruta
                folium.features.GeoJson(json_route, style_function=lambda x: {'color': random.choice(colores)}).add_to(mapa)

                # Obtiene la duración del trayecto
                poi_duration = json_route['features'][0]['properties']['summary']['duration']

                # Guarda la duración del trayecto
                pois['durations'].append(poi_duration)

            # Nos quedamos con la posición de la menor duración
            indice = 0
            if pois['durations']:
                indice = np.argmin(pois['durations'])

            # Para los puntos de interés, se pintan todos del mismo color menos el más cercano con su respectivo globo de texto indicando el tiempo estimado
            for poi in pois['geojson']:
                popup = folium.Popup("Tiempo estimado:\n" + str(int(pois['durations'][pois['geojson'].index(poi)] / 60)) + " minutos " + str(
                    int(round((pois['durations'][pois['geojson'].index(poi)] % 1*60), 2))) + " segundos", max_width=500)
                if pois['geojson'].index(poi) == indice:
                    folium.map.Marker(list(reversed(poi['geometry']['coordinates'])),
                                      icon=folium.Icon(color='lightgreen',
                                                       icon_color='darkblue',
                                                       icon=iconos[cat],
                                                       prefix='fa'
                                                       ), popup=popup).add_to(mapa)
                else:
                    folium.map.Marker(list(reversed(poi['geometry']['coordinates'])),
                                      icon=folium.Icon(color='white',
                                                       icon_color='darkblue',
                                                       icon=iconos[cat],
                                                       prefix='fa'
                                                       ), popup=popup).add_to(mapa)

    

    # Para los pisos, se pintan todos del mismo color menos el que más puntos de interés distintos contenga
    for nombre, piso in pisos.items():
        popup = folium.Popup(nombre, max_width=500)
        if nombre == categorias_por_piso[0][0]:
            folium.Marker(piso['location'],
                          icon=folium.Icon(color='darkpurple',
                                           icon_color='white',
                                           icon='star',
                                           prefix='fa',
                                           ),
                          popup=nombre).add_to(mapa)
        else:
            folium.Marker(piso['location'],
                          icon=folium.Icon(color='darkblue',
                                           icon_color='white',
                                           icon='home',
                                           prefix='fa',
                                           ),
                          popup=popup).add_to(mapa)

    # Restaura la lista de preferencias
    categorias_pi.update(categorias_pi_copia)

    print("Enviando mapa...")
    mapa.save("mapa_pisos.html")
    print("Mapa enviado.")

###BOT DE TELEGRAM
####FUNCIONES
Las funciones que utilizarán los distintos comandos que se implementarán más tarde en este notebook son las siguientes:
* **gen_markup_tiempo()**: se encarga de mostrar los botones que permitirán al usuario el tiempo en el que quiere llegar a sus puntos de interés.
* **gen_markup_prefs()**: se encarga de mostrar los botones que permitirán al usuario activar o desactivar la búsqueda de los distintos puntos de interés.
* **callback_query(...)**: se encarga de manejar la información que le llega al bot a través de los botones que pulsa el usuario y actualizar así los parámetros de búsqueda

In [4]:
bot = telebot.TeleBot("5321518978:AAH5Nbv1DR2GHViQNdBYMscb29MBEz1-6hE", parse_mode=None)

# Crear botones para elegir el tiempo
def gen_markup_tiempo():
    markup = InlineKeyboardMarkup()
    markup.row_width = 3

    btn_5 = InlineKeyboardButton("< 5", callback_data="tiempo_5")
    btn_10 = InlineKeyboardButton("< 10", callback_data="tiempo_10")
    btn_15 = InlineKeyboardButton("< 15", callback_data="tiempo_15")

    markup.add(btn_5, btn_10, btn_15)

    return markup


# Crear botones para elegir los puntos de interés
def gen_markup_prefs():
    markup = InlineKeyboardMarkup()
    markup.row_width = 2

    if preferencias["supermarket"] == 0:
        btn_supermercado = InlineKeyboardButton("Supermercados 🔴", callback_data="supermercados_si")
    else:
        btn_supermercado = InlineKeyboardButton("Supermercados 🟢", callback_data="supermercados_no")

    if preferencias["platform"] == 0:
        btn_parada_bus = InlineKeyboardButton("Paradas de Bus 🔴", callback_data="paradas_si")
    else:
        btn_parada_bus = InlineKeyboardButton("Paradas de Bus 🟢", callback_data="paradas_no")

    if preferencias["bus_station"] == 0:
        btn_estacion_bus = InlineKeyboardButton("Estaciones de Bus 🔴", callback_data="estaciones_si")
    else:
        btn_estacion_bus = InlineKeyboardButton("Estaciones de Bus 🟢", callback_data="estaciones_no")

    if preferencias["hairdresser"] == 0:
        btn_peluqueria = InlineKeyboardButton("Peluquerías 🔴", callback_data="peluquerias_si")
    else:
        btn_peluqueria = InlineKeyboardButton("Peluquerías 🟢", callback_data="peluquerias_no")

    if preferencias["pub"] == 0:
        btn_pub = InlineKeyboardButton("Pubs 🔴", callback_data="pub_si")
    else:
        btn_pub = InlineKeyboardButton("Pubs 🟢", callback_data="pub_no")

    if preferencias["bar"] == 0:
        btn_bar = InlineKeyboardButton("Bares 🔴", callback_data="bar_si")
    else:
        btn_bar = InlineKeyboardButton("Bares 🟢", callback_data="bar_no")

    if preferencias["fast-food"] == 0:
        btn_comida = InlineKeyboardButton("Comida Rápida 🔴", callback_data="comida_si")
    else:
        btn_comida = InlineKeyboardButton("Comida Rápida 🟢", callback_data="comida_no")

    if preferencias["pharmacy"] == 0:
        btn_farmacia = InlineKeyboardButton("Farmacias 🔴", callback_data="farmacia_si")
    else:
        btn_farmacia = InlineKeyboardButton("Farmacias 🟢", callback_data="farmacia_no")

    if preferencias["fitness_centre"] == 0:
        btn_gym = InlineKeyboardButton("Gimnasios 🔴", callback_data="gym_si")
    else:
        btn_gym = InlineKeyboardButton("Gimnasios 🟢", callback_data="gym_no")

    if preferencias["park"] == 0:
        btn_parque = InlineKeyboardButton("Parques 🔴", callback_data="parque_si")
    else:
        btn_parque = InlineKeyboardButton("Parques 🟢", callback_data="parque_no")

    if preferencias["bank"] == 0:
        btn_banco = InlineKeyboardButton("Bancos 🔴", callback_data="banco_si")
    else:
        btn_banco = InlineKeyboardButton("Bancos 🟢", callback_data="banco_no")

    if preferencias["hospital"] == 0:
        btn_hospital = InlineKeyboardButton("Hospitales 🔴", callback_data="hospital_si")
    else:
        btn_hospital = InlineKeyboardButton("Hospitales 🟢", callback_data="hospital_no")

    if preferencias["university"] == 0:
        btn_universidad = InlineKeyboardButton("Universidad 🔴", callback_data="universidad_si")
    else:
        btn_universidad = InlineKeyboardButton("Universidad 🟢", callback_data="universidad_no")

    markup.add(btn_supermercado, btn_parada_bus, btn_estacion_bus, btn_peluqueria, btn_pub, btn_bar,
               btn_comida, btn_farmacia, btn_gym, btn_parque, btn_banco, btn_hospital, btn_universidad)

    return markup


# Manejador para las pulsaciones de los botones
@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    # TIEMPO
    if call.data == "tiempo_5":
        params_iso["range"] = [5 * 60]
        bot.answer_callback_query(call.id, "Tiempo fijado a 5 minutos o menos")
    elif call.data == "tiempo_10":
        params_iso["range"] = [10 * 60]
        bot.answer_callback_query(call.id, "Tiempo fijado a 10 minutos o menos")

    elif call.data == "tiempo_15":
        params_iso["range"] = [15 * 60]
        bot.answer_callback_query(call.id, "Tiempo fijado a 15 minutos o menos")

    # PREFERENCIAS
    if call.data == "supermercados_si":
        preferencias["supermarket"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "supermercados_no":
        preferencias["supermarket"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "paradas_si":
        preferencias["platform"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "paradas_no":
        preferencias["platform"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "estaciones_si":
        preferencias["bus_station"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "estaciones_no":
        preferencias["bus_station"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "peluquerias_si":
        preferencias["hairdresser"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "peluquerias_no":
        preferencias["hairdresser"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "pub_si":
        preferencias["pub"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "pub_no":
        preferencias["pub"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "bar_si":
        preferencias["bar"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "bar_no":
        preferencias["bar"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "comida_si":
        preferencias["fast-food"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "comida_no":
        preferencias["fast-food"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "farmacia_si":
        preferencias["pharmacy"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "farmacia_no":
        preferencias["pharmacy"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "gym_si":
        preferencias["fitness_centre"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "gym_no":
        preferencias["fitness_centre"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "parque_si":
        preferencias["park"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "parque_no":
        preferencias["park"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "banco_si":
        preferencias["bank"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "banco_no":
        preferencias["bank"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "hospital_si":
        preferencias["hospital"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "hospital_no":
        preferencias["hospital"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data == "universidad_si":
        preferencias["university"] = 1
        bot.answer_callback_query(call.id, "Preferencia activada")
    elif call.data == "universidad_no":
        preferencias["university"] = 0
        bot.answer_callback_query(call.id, "Preferencia desactivada")

    if call.data.startswith("tiempo") != True:
        bot.send_message(
            call.message.chat.id, "Elige tus puntos de interés:", reply_markup=gen_markup_prefs())
      

####COMANDOS
Los comandos que harán uso de las funciones implementadas anteriormente son los siguientes:
* **/start**: mandará un mensaje de bienvenida al iniciar el bot o al escribir "*/start*".
* **/ayuda**: mandará un mensaje con información sobre cómo utilizar el bot.
* **/tiempo**: mostrará una serie de botones para que el usuario elija el tiempo en el que quiere llegar a sus puntos de interés.
* **/preferencias**: mostrará una serie de botones para que el usuario active o desactive los puntos de interés que quiere encontrár cerca de su piso.
* **/buscarpiso**: inicia la búsqueda de pisos según las preferencias fijadas por el usuario.

In [5]:
# Comando start que muestra mensaje de bienvenida
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(
        message, '¡Bienvenido! Este bot te ayudará a encontrar tu piso ideal en Cáceres! Utiliza /ayuda para más información.')


# Comando ayuda que muestra los comandos disponibles
@bot.message_handler(commands=['ayuda'])
def info(message):
    bot.reply_to(message, 'Comandos disponibles:\n /tiempo - Elige en cuántos minutos quieres encontrar tus puntos de interés\n /preferencias - Elige qué puntos de interés quieres cerca de tu piso.\n /buscarpiso - Inicia la búsqueda de pisos.\n')


# Comando tiempo que permite al usuario elegir los minutos en los que quiere llegar a sus puntos de interés
@bot.message_handler(commands=['tiempo'])
def tiempo(message):
    print("Cambiando tiempo")
    bot.send_message(message.chat.id, "Elige en cuántos minutos quieres llegar a tus puntos de interés:",
                     reply_markup=gen_markup_tiempo())


# Comando preferencias que permite al usuario elegir sus puntos de interés
@bot.message_handler(commands=['preferencias'])
def prefs(message):
    print("Cambiando preferencias")
    bot.send_message(message.chat.id, "Elige tus puntos de interés:",
                     reply_markup=gen_markup_prefs())


# Comando buscarpiso que inicia la búsqueda en función de los parametros fijados por el usuario
@bot.message_handler(commands=['buscarpiso'])
def buscar_piso(message, preferencias=preferencias, pisos=pisos, params_iso=params_iso, parametros_pi=parametros_pi, categorias_pi=categorias_pi, iconos=iconos, parametros_ruta=parametros_ruta):
    print("Generando mapa...")
    bot.send_message(message.chat.id, "Buscando pisos... Esto puede tardar un poco...")
    buscar_pisos(preferencias, pisos, params_iso, parametros_pi, categorias_pi, iconos, parametros_ruta)
    mapa = open('./mapa_pisos.html')
    bot.send_message(message.chat.id, "¡Aquí tienes tu mapa con tus preferencias!")
    bot.send_document(message.chat.id, mapa)

####PUESTA EN MARCHA DEL BOT
Para finalizar el código, se inicia el bot con la función *infinity_polling(...)*. A partir de este momento, el bot estará a la espera de mensajes que contengan los comandos de los usuarios.

In [ ]:
# Inicia el bot de Telegram
print("Bot Iniciado...")
bot.infinity_polling(interval=0, timeout=120)
print("Bot Detenido.")

Bot Iniciado...
